# Homework 2 Part 1

## Course Name: Large Language Models
#### Lecturers: Dr. Soleimani, Dr. Rohban, Dr. Asgari

---

#### Notebooks Supervised By: MohammadAli SadraeiJavaheri, Omid Ghahroodi
#### Notebook Prepared By: Omid Ghahroodi

**Contact**: Ask your questions in Quera

---

### Instructions:
- Complete all exercises presented in this notebook.
- Ensure you run each cell after you've entered your solution.
- After completing the exercises, save the notebook and <font color='red'>follow the submission guidelines provided in the PDF.</font>


---

**Note**: Replace the placeholders (between <font color="green">`## Your code begins ##`</font> and <font color="green">`## Your code ends ##`</font>) with the appropriate details.


## 1. Introduction

In this notebook, you will learn about data preprocessing for training a large language model and tokenizers.

In this notebook first, you download the raw data, then implement a pipeline for preprocessing this data. Finally, you train the three different types of tokenizers that you learned in the class on your data and compare them.

Answer the questions marked in green font.


In [1]:
# Note: Do NOT make changes to this block.

!pip install transformers
!pip install fasttext
!pip install hazm


from IPython.display import clear_output
from huggingface_hub import hf_hub_download
from tqdm.notebook import tqdm
import numpy as np
import random
import transformers
import gzip
import json
import fasttext
from hazm import *

normalizer = Normalizer()

SEED=21

np.random.seed(SEED)
random.seed(SEED)

clear_output()

### Download Raw Data

In this section, download the raw data.

In [3]:
# Note: Do NOT make changes to this block.

!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datasets/allenai/c4
%cd c4
!git lfs pull --include "c4-fa.tfrecord-00000-of-01024.json.gz"
%cd ..

file_path = './c4/multilingual/c4-fa.tfrecord-00000-of-01024.json.gz'

with gzip.open(file_path, 'rb') as f:
    file_content = f.read().decode('utf8')

data = file_content.split('\n')

text_data = []

for i in data:
    if i!='':
        text_data.append(json.loads(i)['text'])

!wget https://raw.githubusercontent.com/roshan-research/hazm/master/hazm/data/stopwords.dat
with open('stopwords.dat', 'r') as f:
    stop_words = f.read()
stop_words = stop_words.rstrip()
stop_words = stop_words.split('\n')

clear_output()

[WinError 2] The system cannot find the file specified: 'c4'
c:\Users\jrazi\Desktop\repo\llm-assignments-fall-2023\HW2


'GIT_LFS_SKIP_SMUDGE' is not recognized as an internal or external command,
operable program or batch file.


c:\Users\jrazi\Desktop\repo\llm-assignments-fall-2023


FileNotFoundError: [Errno 2] No such file or directory: './c4/multilingual/c4-fa.tfrecord-00000-of-01024.json.gz'

### Statistics (5 points)

In this section, you should report several statistical characteristics of the data:
- Average length of the documents
- Largest length of the documents
- Smallest length of the documents
- The number of words in the data
- The most frequent word

In [ ]:
## Your code begins ##

## Your code ends ##

In [ ]:
# Note: Do NOT make changes to this block.
# Note: Do NOT change orginal_text_data

orginal_text_data = text_data[:]
len(orginal_text_data)


### Low-quality Examples

In this section, several examples of poor-quality data from each category are provided. At the end of the preprocessing pipeline, you must remove such data from the original dataset to obtain high-quality data.

In [ ]:
# Note: Do NOT make changes to this block.

non_persian_example = orginal_text_data[9]
print(non_persian_example)


In [ ]:
# Note: Do NOT make changes to this block.

short_length_example = orginal_text_data[1494]
print(short_length_example)


In [ ]:
# Note: Do NOT make changes to this block.

mean_length_word_example = orginal_text_data[250]
print(mean_length_word_example)


In [ ]:
# Note: Do NOT make changes to this block.

symbol_ration_example = orginal_text_data[15018]
print(symbol_ration_example)


In [ ]:
# Note: Do NOT make changes to this block.

alphabet_example = orginal_text_data[1]
print(alphabet_example)


In [ ]:
# Note: Do NOT make changes to this block.

stopword_example = orginal_text_data[74]
print(stopword_example)

## 2. Preprocessing (85 points)

In this part, you have to implement the pre-processing pipeline. This pipeline consists of the following sections:

1- Language Identification: In this section, you need to identify the language of the data using FastText and remove any data that is not in Farsi.

2- Length Filter: It is likely that examples that are shorter or longer than one length do not contain complete sentences or textual data. In this section, you should remove this data.

3- Mean Word Length: In high quality text, words should have an average number of characters within a certain range. you should find the average word length in each sample and exclude samples that fall outside the range.

4- Symbol Ratio: To ensure that a text is of good quality for training a model, the ratio of symbol count to word count should not exceed the threshold. In this task, you should calculate the ratio of symbol count to word count in any samples and filter out any samples with a ratio above a certain threshold.

5- Alphabetic Filter: In this part, you need to find the ratio of the number of words that contain at least one letter from the alphabet to the ratio of total number of words. If this ratio is below a certain limit, you should remove the sample.

6- Stop Word Filter: Find the number of stop words in a sample. If it's below a limit, delete the sample.

In [ ]:
# Ensure that each sample is labeled either as "__label__pes_Arab" or "__label__prs_Arab"

# You must find the right hyperparameters for these!!!
## Your code begins ##
length_filter_min = None
length_filter_max = None
word_length_filter_min = None
word_length_filter_max = None
symbol_filter_max_ratio = None
alphabetic_filter_min_ratio = None
stop_words_filter_min_count = None
## Your code ends ##
##################################################

def language_filter(text_data):
    model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
    model = fasttext.load_model(model_path)
    return_value = []
    for text_row in tqdm(text_data):
        text = text_row.replace("\n", " ")
        ## Your code begins ##
        lang = None
        ## Your code ends ##
        if lang in ['__label__pes_Arab', '__label__prs_Arab']:
            return_value.append(text_row)
    return return_value


# Keep samples that have a word count between "min_length" and "max_length"
def length_filter(text_data, min_length=length_filter_min, max_length=length_filter_max):
    return_value = []
    for text_row in tqdm(text_data):
        ## Your code begins ##
        length = None
        ## Your code ends ##
        if length >= min_length and length <= max_length:
            return_value.append(text_row)
    return return_value


# Keep samples that have a mean word length between "min_length" and "max_length"
def word_length_filter(text_data, min_length=word_length_filter_min, max_length=word_length_filter_max):
    return_value = []
    for text_row in tqdm(text_data):
        ## Your code begins ##
        words = None
        total_word_length = None
        mean_word_length = None
        ## Your code ends ##
        if min_length <= mean_word_length <= max_length:
            return_value.append(text_row)
    return return_value

# Calculate the ratio of symbol count to total word count for each sample then
# keep samples that have a ratio lower than the "max_ratio"
def symbol_filter(text_data, max_ratio=symbol_filter_max_ratio):
    symbols = ["#", "*", "...", "@", "$", "&"]
    return_value = []
    for text_row in tqdm(text_data):
        ## Your code begins ##
        words = None
        total_symbols = 0
        for j in symbols:
            total_symbols += None
        total_words = None
        ## Your code ends ##
        symbol_to_word_ratio = total_symbols / total_words if total_words > 0 else 0
        if symbol_to_word_ratio <= max_ratio:
            return_value.append(text_row)
    return return_value



# Calculate the ratio of alphabetical-word count to total word count for each sample then
# keep samples that have a ratio higher than the "min_ratio"
def alphabetic_filter(text_data, min_ratio=alphabetic_filter_min_ratio):
    return_value = []
    for text_row in tqdm(text_data):
        ## Your code begins ##
        words = None
        alphabetic_word_count = None  # you can use str.isalpha
        total_words = None
        ## Your code ends ##
        alphabetic_ratio = alphabetic_word_count / total_words if total_words > 0 else 0
        if alphabetic_ratio >= min_ratio:
            return_value.append(text_row)
    return return_value


# Keep samples that have a stop word count higher than the "min_count"
# Stop words list provided in the "stop_words" variable
def stop_words_filter(text_data, min_count=stop_words_filter_min_count):
    return_value = []
    for text_row in tqdm(text_data):
        ## Your code begins ##
        words = None
        stop_word_count = None
        ## Your code ends ##
        if stop_word_count >= min_count:
            return_value.append(text_row)
    return return_value

### Sanity Check

In this cell, an simple check is done on the examples.

In [ ]:
# Note: Do NOT make changes to this block.

good_example = orginal_text_data[150]

assert (language_filter([non_persian_example, good_example])==[good_example])
assert (length_filter([short_length_example, good_example])==[good_example])
assert (word_length_filter([mean_length_word_example, good_example])==[good_example])
assert (symbol_filter([symbol_ration_example, good_example])==[good_example])
assert (alphabetic_filter([alphabet_example, good_example])==[good_example])
assert (stop_words_filter([stopword_example, good_example])==[good_example])


### Normalization

In this cell, normalize the filtered texts.

Using `normalizer.normalize`.

In [ ]:
def normalize(text_data):
    return = [
        ## Your code begins ##
        None
        ## Your code ends ##
        for text_row in tqdm(text_data)
    ]

### Pipeline

Run your preprocessing pipeline in this section.

In the input, give all the raw data, and in the output,  find high-quality data, and finally, normalize them.

Find the hyperparameters of the preprocessing pipeline so that at the end 60% of raw data remains. Hyperparameter list is:
- length_filter_min
- length_filter_max
- word_length_filter_min
- word_length_filter_max
- symbol_filter_max_ratio
- alphabetic_filter_min_ratio
- stop_words_filter_min_count

In [ ]:
total = len(text_data)

print(f'Total number of raw data: {total}')

text_data = language_filter(text_data)
print(f'{(len(text_data)/total)*100}% of raw data remained after language filtering')

text_data = length_filter(text_data)
print(f'{(len(text_data)/total)*100}% of raw data remained after length filtering')

text_data = word_length_filter(text_data)
print(f'{(len(text_data)/total)*100}% of raw data remained after mean word length filtering')

text_data = symbol_filter(text_data)
print(f'{(len(text_data)/total)*100}% of raw data remained after symbol ratio filtering')

text_data = alphabetic_filter(text_data)
print(f'{(len(text_data)/total)*100}% of raw data remained after alphabet filtering')

text_data = stop_words_filter(text_data)
print(f'{(len(text_data)/total)*100}% of raw data remained after stop word filtering')

text_data = normalize(text_data)


### Save

In [ ]:
# Note: Do NOT make changes to this block.

with open('train.txt', 'w', encoding="utf-8") as f:
    f.write('\n'.join(text_data))

with open('raw_text.txt', 'w', encoding="utf-8") as f:
    f.write('\n'.join(orginal_text_data))

# 3. Tokenizer (10 points)

In this section, you will train tokenizers for cleaned and raw data. The training code is provided. 

Finally, compare the output of the tokenizers.

### WordPiece Tokenizer

In [ ]:
# Note: Do NOT make changes to this block.

from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer_WordPiece = Tokenizer(models.WordPiece(unk_token="[UNK]"))

tokenizer_WordPiece.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.StripAccents()]
)

tokenizer_WordPiece.pre_tokenizer = pre_tokenizers.Sequence(
    [pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()]
)

print(tokenizer_WordPiece.pre_tokenizer.pre_tokenize_str("این یک تست است!!!"))

special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)
tokenizer_WordPiece.train(["train.txt"], trainer=trainer)
tokenizer_WordPiece.save("tokenizer_WordPiece.json")

In [ ]:
# Note: Do NOT make changes to this block.

tokenizer_WordPiece2 = Tokenizer(models.WordPiece(unk_token="[UNK]"))

tokenizer_WordPiece2.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.StripAccents()]
)

tokenizer_WordPiece2.pre_tokenizer = pre_tokenizers.Sequence(
    [pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()]
)


special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)
tokenizer_WordPiece2.train(["raw_text.txt"], trainer=trainer)
tokenizer_WordPiece2.save("tokenizer_WordPiece2.json")


In [ ]:
encoding = tokenizer_WordPiece.encode(text_data[0])
print(encoding.tokens)
encoding = tokenizer_WordPiece2.encode(text_data[0])
print(encoding.tokens)

### BPE Tokenizer

In [ ]:
# Note: Do NOT make changes to this block.

tokenizer_BPE = Tokenizer(models.BPE())
tokenizer_BPE.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
print(tokenizer_BPE.pre_tokenizer.pre_tokenize_str("این یک تست است!!!"))

trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
tokenizer_BPE.train(["train.txt"], trainer=trainer)

tokenizer_BPE.save("tokenizer_BPE.json")


In [ ]:
# Note: Do NOT make changes to this block.

tokenizer_BPE2 = Tokenizer(models.BPE())
tokenizer_BPE2.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
tokenizer_BPE2.train(["raw_text.txt"], trainer=trainer)

tokenizer_BPE2.save("tokenizer_BPE2.json")


In [ ]:
encoding = tokenizer_BPE.encode(text_data[0])
print(encoding.tokens)
encoding = tokenizer_BPE2.encode(text_data[0])
print(encoding.tokens)

### Unigram Tokenizer

In [ ]:
# Note: Do NOT make changes to this block.

tokenizer_unigram = Tokenizer(models.Unigram())

tokenizer_unigram.normalizer = normalizers.Sequence(
    [
        normalizers.NFKD(),
        normalizers.StripAccents()
    ]
)

tokenizer_unigram.pre_tokenizer = pre_tokenizers.Metaspace()

tokenizer_unigram.pre_tokenizer.pre_tokenize_str("این یک تست است!!!")

special_tokens = ["<cls>", "<sep>", "<unk>", "<pad>", "<mask>", "<s>", "</s>"]
trainer = trainers.UnigramTrainer(
    vocab_size=25000, special_tokens=special_tokens, unk_token="<unk>"
)
tokenizer_unigram.train(["train.txt"], trainer=trainer)


tokenizer_unigram.save("tokenizer_unigram.json")


In [ ]:
# Note: Do NOT make changes to this block.

tokenizer_unigram2 = Tokenizer(models.Unigram())

tokenizer_unigram2.normalizer = normalizers.Sequence(
    [
        normalizers.NFKD(),
        normalizers.StripAccents()
    ]
)

tokenizer_unigram2.pre_tokenizer = pre_tokenizers.Metaspace()

special_tokens = ["<cls>", "<sep>", "<unk>", "<pad>", "<mask>", "<s>", "</s>"]
trainer = trainers.UnigramTrainer(
    vocab_size=25000, special_tokens=special_tokens, unk_token="<unk>"
)
tokenizer_unigram2.train(["raw_text.txt"], trainer=trainer)


tokenizer_unigram2.save("tokenizer_unigram2.json")


In [ ]:
encoding = tokenizer_unigram.encode(text_data[0])
print(encoding.tokens)
encoding = tokenizer_unigram2.encode(text_data[0])
print(encoding.tokens)

### Analyze

In this part, WordPiece tokenizer and Unigram tokenizer are used on the data and the number of generated tokens and their intersection are displayed.

According to this section, answer the following questions.

<font color='green'>

1- Compare the output of tokenizers.

2- Why is the BPE output unreadable?

3- Analyze the intersection between WorkPiece tokens and Unigram tokens.

</font>

In [ ]:
unigram_decoding = []
wordpiece_decoding = []

for i in tqdm(text_data):
    wordpiece_decoding.append(tokenizer_WordPiece.encode(i).tokens)
    unigram_decoding.append(tokenizer_unigram.encode(i).tokens)


In [ ]:
import itertools

print(f'Number of tokens generated by unigram: {len(list(itertools.chain(*unigram_decoding)))}')
print(f'Number of tokens generated by wordpiece: {len(list(itertools.chain(*wordpiece_decoding)))}')

print(f'Number of unique tokens generated by unigram: {len(set(itertools.chain(*unigram_decoding)))}')
print(f'Number of unique tokens generated by wordpiece: {len(set(itertools.chain(*wordpiece_decoding)))}')

print(f'Number of intersection tokens: {len(list(set(itertools.chain(*unigram_decoding)) & set(itertools.chain(*wordpiece_decoding))))}')
